# Data Wrangling & Exploration

This notebook captueres the data wrangling and exploration of image stabalization data sets. We have accress to multiple data sets.  They have been gathered to insure quicker access to the original data sets that are distriputed across the internet. It also means the data is under a type of source control, frozen at the version I pulled. This is both good and bad because it will be dificult to see modifications that I may want. But we will cross that bridge if it comes up.

## Read the StabNet Data Set

A copy of the StabNet Data set is frozen on an S3.  I will read it from there extract the data.

In [3]:
%%time

import os
import boto3

project_dir = os.getcwd()

data_dir = 'Data'
if not os.path.isdir(data_dir):
    os.mkdir(data_dir)
os.chdir(data_dir)

s3 = boto3.client('s3')
s3.download_file('madat-machine-learning-data', 'Mark/capstone-project/DeepStab.zip', 'DeepStab.zip')

from zipfile import ZipFile

with ZipFile('DeepStab.zip', 'r') as zipObj:
    zipObj.extractall()
    
os.chdir('..')
os.getcwd()

'/home/sagemaker-user/hand-tracking-stabilization'

From the Readme.txt:

Here we provide the DeepStab dataset of the pre-print paper 'Deep Online Video Stabilization'.
The dataset contains 1280x720 paired videos, from each pair one is stable and the other is unstable.
The videos are organized in two corresponding folders ./stable and ./unstable

In [4]:
import os.path

stable_avis = 'Data/DeepStab/stable'
unstable_avis = 'Data/DeepStab/unstable'
number_stable_avis = len([name for name in os.listdir(stable_avis) if os.path.isfile(os.path.join(stable_avis, name))])
number_unstable_avis = len([name for name in os.listdir(unstable_avis) if os.path.isfile(os.path.join(unstable_avis, name))])

number_stable_avis , number_unstable_avis

(61, 61)

In [7]:
# make sure libGL is on the instance
!/opt/conda/bin/python -m pip install --upgrade pip
!pip install opencv-python

!sudo apt update
!sudo apt -y install libgl1-mesa-glx

Defaulting to user installation because normal site-packages is not writeable
sudo: unable to resolve host sagemaker-jumpstart-d-ml-t3-medium-587ae20b053ee75476e1e39e9fae
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease   
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]      
Get:4 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1421 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]    
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [2730 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2294 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [543 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates/multiverse amd64 Packages [34.4 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 Packages [2196 kB]
Get:

Verify the stable and unstable frame counts match and that they are of the correct shape.

In [8]:
%%time

import cv2
import os

stable_avi_frame_count = 0
for file_number in range(1,number_stable_avis+1):
           
    vidcap = cv2.VideoCapture(os.path.join(stable_avis, f'{file_number}.avi'))
    success,image = vidcap.read()

    while success:
        if image.shape != (720,1280,3):
            print(f'Invalid Image shape: {image.shape}')
        stable_avi_frame_count += 1
                  
        success,image = vidcap.read()

unstable_avi_frame_count = 0
for file_number in range(1,number_unstable_avis+1):
           
    vidcap = cv2.VideoCapture(os.path.join(unstable_avis, f'{file_number}.avi'))
    success,image = vidcap.read()

    while success:
        if image.shape != (720,1280,3):
            print(f'Invalid Image shape: {image.shape}')
        unstable_avi_frame_count += 1
     
        success,image = vidcap.read()
        
stable_avi_frame_count, unstable_avi_frame_count
               

(43585, 43585)

## Read the Dataset used for Evaluation Deep Learning Based Video Stabilization System Frameworks

An important aspect of this data set is it contains translation data for the labeled stabel data set. This is seen as extremely useful for understanding and evaluating the performance of the model.

This is going to take some time. Tar is extreamely slow due to its serial nature and compression. The data set load is in the order of 55 minutes. 


In [12]:
%%time

import os
import boto3
import tarfile

os.chdir(project_dir)
print(os.getcwd())

data_dir = 'Data/EvaluationDataSet'
if not os.path.isdir(data_dir):
    os.mkdir(data_dir)
os.chdir(data_dir)

s3 = boto3.client('s3')
s3.download_file('madat-machine-learning-data', 'Mark/capstone-project/Dataset.tar.gz', 'DataSet.tar.gz')
s3.download_file('madat-machine-learning-data', 'Mark/capstone-project/Dataset-3.tar.gz', 'DataSet-3.tar.gz')

tar = tarfile.open('DataSet.tar.gz')
tar.extractall()
tar.close()

tar = tarfile.open('DataSet-3.tar.gz')
tar.extractall()
tar.close()

os.chdir('../..')
os.getcwd()

/home/sagemaker-user/hand-tracking-stabilization
CPU times: user 3min 16s, sys: 3min 12s, total: 6min 28s
Wall time: 53min 23s


'/home/sagemaker-user/hand-tracking-stabilization'